[View in Colaboratory](https://colab.research.google.com/github/clee1994/DLclass/blob/master/HW02_Convolution.ipynb)

In [31]:
import numpy as np
from scipy import signal as signal
import matplotlib.pyplot as plt
import time
import tensorflow  as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
def tf2dConv(matrix_reshaped,kernel_reshaped, gpu=False):
  config = tf.ConfigProto(allow_soft_placement = True)
  config.gpu_options.allow_growth = True
  
  if gpu:
    device_string = '/gpu:0'
  else:
    device_string = '/cpu:0'

  with tf.device(device_string):
    
    input_matrix =  tf.Variable(matrix_reshaped.astype(float))
    kernel_tf = tf.constant(kernel_reshaped.astype(float))

    convoluted_matrix = tf.nn.convolution(input=input_matrix,filter=kernel_tf,strides=[1,1],padding="VALID")
    init = tf.global_variables_initializer()

    with tf.Session(config=config) as sess:
      sess.run(init)
      time1 = time.time()
      output_matrix = sess.run(convoluted_matrix)
      time2 = time.time()
  return output_matrix[0],(time2-time1)



In [0]:
def padWith(vector, pad_width, iaxis, kwargs):
  pad_value = kwargs.get('padder', 10)
  vector[:pad_width[0]] = pad_value
  vector[-pad_width[1]:] = pad_value
  return vector

In [0]:
def joshi2dpatch(patch,kernel):
  flippedKernel = np.fliplr(np.flipud(kernel))
  appliedKernel = np.multiply(patch,flippedKernel)
  outPixel = np.sum(np.sum(appliedKernel))
  
  return outPixel

In [0]:
def joshi2dConv(inMatrix,kernel):
  
  kSize = kernel.shape
  pSize = inMatrix.shape
  outputH = (pSize[0]-kSize[0])+1
  outputW = (pSize[1]-kSize[1])+1
  outputPatch = np.zeros([outputH,outputW])
  
  
  for outer in range(outputH):
    for inner in range(outputW):
      outputPatch[outer,inner] = joshi2dpatch(inMatrix[outer:(outer+kSize[0]),inner:(inner+kSize[1])],kernel) 
  
  return outputPatch

In [0]:
def runConvs(matrix, kernel):
  iters = 100
  
  
  mshape = matrix.shape
  kshape = kernel.shape
  temp_mat = np.repeat(matrix,iters)
  matrix_reshaped = temp_mat.reshape(iters, mshape[0], mshape[1], 1)
  kernel_reshaped = kernel.reshape(kshape[0], kshape[1], 1, 1)
  
  
  results = np.ones([iters,4])*np.nan
  
  
  #tensorflow with CPU/GPU
  testConvTf, tftime = tf2dConv(matrix_reshaped,kernel_reshaped,gpu=False)
  results[0,2] = 1/(tftime/100)
  testConvTf, tftime = tf2dConv(matrix_reshaped,kernel_reshaped,gpu=True)
  results[0,3] = 1/(tftime/100)
  
  for i in range(iters):
    #Joshi
    time1 = time.time()
    testConvJoshi = joshi2dConv(matrix,kernel)
    time2 = time.time()
    results[i,0] = 1/(time2-time1)
    

    #Scipy
    time1 = time.time()
    testConvScipy = signal.convolve2d(matrix,kernel,mode='valid')
    time2 = time.time()
    results[i,1] = 1/(time2-time1)
    


    
    
  
  print('Joshi Convolutions per second: {:.3f} +/- {:.3f}'.format(np.average(results[:,0]), np.std(results[:,0])))
  print('Scipy Convolutions per second: {:.3f} +/- {:.3f}'.format(np.average(results[:,1]), np.std(results[:,1])))
  print('Tensorflow Convolutions per second: {:.3f} '.format(results[0,2]))
  print('Tensorflow with GPU Convolutions per second: {:.3f} '.format(results[0,3]))
  

In [57]:
if __name__=='__main__':
  
  ksizes = [3,5,7]
  msizes = [28,32]
 
  
  for i in ksizes:
    for j in msizes:
      print("# matrix size: {}, kernel size: {}".format(i,j))
      kernel = np.eye(i)
      matrix = np.random.randn(j,j)*100
      runConvs(matrix,kernel)
      print("-"*25)
  
  
  



# matrix size: 3, kernel size: 28
Joshi Convolutions per second: 117.463 +/- 3.997
Scipy Convolutions per second: 13313.248 +/- 2407.592
Tensorflow Convolutions per second: 407.700 
Tensorflow with GPU Convolutions per second: 402.082 
-------------------------
# matrix size: 3, kernel size: 32
Joshi Convolutions per second: 85.437 +/- 7.282
Scipy Convolutions per second: 9587.913 +/- 2436.348
Tensorflow Convolutions per second: 384.337 
Tensorflow with GPU Convolutions per second: 364.839 
-------------------------
# matrix size: 5, kernel size: 28
Joshi Convolutions per second: 130.896 +/- 11.136
Scipy Convolutions per second: 10108.983 +/- 1933.915
Tensorflow Convolutions per second: 373.466 
Tensorflow with GPU Convolutions per second: 409.620 
-------------------------
# matrix size: 5, kernel size: 32
Joshi Convolutions per second: 97.810 +/- 7.991
Scipy Convolutions per second: 7801.327 +/- 1631.578
Tensorflow Convolutions per second: 363.231 
Tensorflow with GPU Convolutions pe